In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import  LinearRegressionModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassificationModel,DecisionTreeClassificationModel
from pyspark.ml import PipelineModel
mount_point = "/mnt/capstonejs"

## Load the Trained models

In [0]:
lr_model_path = dbutils.widgets.get("lr_model_path")
lr_model = LinearRegressionModel.load(lr_model_path)

dt_model_path = dbutils.widgets.get("dt_model_path")
dt_model = DecisionTreeClassificationModel.load(dt_model_path)

dt_model_path2 = dbutils.widgets.get("dt_best_path")
dt_best = DecisionTreeClassificationModel.load(dt_model_path2)

rf_model_path = dbutils.widgets.get("rf_model_path")
rf_model = RandomForestClassificationModel.load(rf_model_path)

## Load the Processed data

In [0]:

test_data_path = "/mnt/capstonejs/dataset/processed/test"
test_data = spark.read.parquet(test_data_path)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1890957258638476>, line 2
      1 test_data_path = "/mnt/capstonejs/dataset/processed/test"
----> 2 test_data = spark.read.parquet(test_data_path)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:544, in DataFrameReader.parquet(self, *paths, **options)
    533 int96RebaseMode = options.get("int96RebaseMode", None)
    534 self._set_opts(
    535     mergeSchema=mergeSchema,
    536     pathGlobFilter=pathGlobFilter,
   (...)
    541     int96RebaseM

In [0]:
evaluator1 = RegressionEvaluator(labelCol="quality", predictionCol="prediction", metricName="mse")
evaluator2 = MulticlassClassificationEvaluator(labelCol="quality")

In [0]:
mse={}
acc={}
prec={}
recall={}
f1={}

## LR

In [0]:
lr_pred = lr_model.transform(test_data)

mse["lr"] = evaluator1.evaluate(lr_pred)
acc["lr"] = evaluator2.evaluate(lr_pred,{evaluator2.metricName: "accuracy"})
prec["lr"]= evaluator2.evaluate(lr_pred,{evaluator2.metricName: "weightedPrecision"})
recall["lr"]= evaluator2.evaluate(lr_pred,{evaluator2.metricName: "weightedRecall"})
f1["lr"] = evaluator2.evaluate(lr_pred, {evaluator2.metricName: "f1"})


## DT

In [0]:
dt_pred = dt_model.transform(test_data)

mse["dt"] = evaluator1.evaluate(dt_pred)
acc["dt"] = evaluator2.evaluate(dt_pred,{evaluator2.metricName: "accuracy"})
prec["dt"]= evaluator2.evaluate(dt_pred,{evaluator2.metricName: "weightedPrecision"})
recall["dt"]= evaluator2.evaluate(dt_pred,{evaluator2.metricName: "weightedRecall"})
f1["dt"] = evaluator2.evaluate(dt_pred, {evaluator2.metricName: "f1"})

In [0]:
dt_pred2 = dt_best.transform(test_data)

mse["dt_best"] = evaluator1.evaluate(dt_pred2)
acc["dt_best"] = evaluator2.evaluate(dt_pred2,{evaluator2.metricName: "accuracy"})
prec["dt_best"]= evaluator2.evaluate(dt_pred2,{evaluator2.metricName: "weightedPrecision"})
recall["dt_best"]= evaluator2.evaluate(dt_pred2,{evaluator2.metricName: "weightedRecall"})
f1["dt_best"] = evaluator2.evaluate(dt_pred2, {evaluator2.metricName: "f1"})

## RF

In [0]:
rf_pred = rf_model.transform(test_data)

mse["rf"] = evaluator1.evaluate(rf_pred)
acc["rf"] = evaluator2.evaluate(rf_pred,{evaluator2.metricName: "accuracy"})
prec["rf"]= evaluator2.evaluate(rf_pred,{evaluator2.metricName: "weightedPrecision"})
recall["rf"]= evaluator2.evaluate(rf_pred,{evaluator2.metricName: "weightedRecall"})
f1["rf"] = evaluator2.evaluate(rf_pred, {evaluator2.metricName: "f1"})

In [0]:
import json
Score_path="/dbfs/mnt/capstonejs/scores"

with open(Score_path, "w") as file:
    file.write("MSE:")
    json.dump(mse,file)
    file.write("\nAccuracy")
    json.dump(acc,file)
    file.write("\nPrecision")
    json.dump(prec,file)
    file.write("\nRecall")
    json.dump(recall,file)
    file.write("\nF1-score")
    json.dump(f1,file)

In [0]:
dbutils.fs.unmount(mount_point)

/mnt/capstonejs has been unmounted.


True